<a href="https://colab.research.google.com/github/AndrewstheBuilder/VideoSemanticSearch_JiuJitsu/blob/main/alphapose_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
# 2. Get AlphaPose
!git clone https://github.com/MVIG-SJTU/AlphaPose.git
!cd AlphaPose


Cloning into 'AlphaPose'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 2749 (delta 4), reused 1 (delta 0), pack-reused 2739 (from 1)
Receiving objects: 100% (2749/2749), 118.82 MiB | 37.40 MiB/s, done.
Resolving deltas: 100% (1379/1379), done.


In [3]:
!export PATH=/usr/local/cuda/bin/:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
!pip install cython
!sudo apt-get install libyaml-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libyaml-doc
The following NEW packages will be installed:
  libyaml-dev
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libyaml-dev amd64 0.2.2-1build2 [62.8 kB]
Fetched 62.8 kB in 1s (118 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libyaml-dev:amd64.
(Read

In [4]:
%cd /content/AlphaPose/
!python3 setup.py build develop --user

/content/AlphaPose
Compiling detector/nms/src/soft_nms_cpu.pyx because it changed.
[1/1] Cythonizing detector/nms/src/soft_nms_cpu.pyx
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:85: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/alphapose
copying alphapose/__init__.py -> build/lib.linux-x86_64-cpython-310/alphapose
copying alphapose/version.py -> build/lib.linux-x86_64-cpython-310/alphapose
copying alphapose/opt.py -> build/lib.linux-x86_64-cp

In [5]:
# Install Yolo3 Weights
!pip install gdown
import gdown
# https://drive.usercontent.google.com/download?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC&export=download&authuser=0

file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'  # Replace with the actual file ID
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output="yolov3-spp.weights", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
From (redirected): https://drive.google.com/uc?export=download&id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC&confirm=t&uuid=46f16836-9e4b-412b-bc6e-295005df6917
To: /content/AlphaPose/yolov3-spp.weights
100%|██████████| 252M/252M [00:01<00:00, 134MB/s]


'yolov3-spp.weights'

In [6]:
# Move yolov3 weights into appropriate directory
!mkdir ./detector/yolo/data
!mv yolov3-spp.weights ./detector/yolo/data

In [7]:
# Download and move baseline Pose Model into right directory
# https://drive.google.com/file/d/1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw/view
file_id = '1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw'  # Replace with the actual file ID
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output="simple_res50_256x192.pth", quiet=False)
!mv simple_res50_256x192.pth ./pretrained_models

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw
From (redirected): https://drive.google.com/uc?export=download&id=1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw&confirm=t&uuid=55440624-2c36-45c6-a632-53fc229857ee
To: /content/AlphaPose/simple_res50_256x192.pth
100%|██████████| 136M/136M [00:00<00:00, 140MB/s]


In [8]:
# Download and move Pose tracker model into right directory
# https://drive.google.com/file/d/1myNKfr2cXqiHZVXaaG8ZAq_U2UpeOLfG/view
file_id = '1myNKfr2cXqiHZVXaaG8ZAq_U2UpeOLfG'  # Replace with the actual file ID
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output="osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth", quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1myNKfr2cXqiHZVXaaG8ZAq_U2UpeOLfG
To: /content/AlphaPose/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth
100%|██████████| 17.3M/17.3M [00:00<00:00, 27.4MB/s]


'osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth'

In [18]:
!mkdir trackers/weights
!mv osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth ./trackers/weights/

mkdir: cannot create directory ‘trackers/weights’: File exists


In [10]:
!pip install cython-bbox

  Preparing metadata (setup.py) ... done
  Created wheel for cython-bbox: filename=cython_bbox-0.1.5-cp310-cp310-linux_x86_64.whl size=99110 sha256=135e2d0d4f8ef50a473e1958f2c0d64ca10b0e86446c77661a9b39580a031c98
  Stored in directory: /root/.cache/pip/wheels/c0/b7/68/bab98b7180cda501101a57fb7d36884218ad45ec60c27cd679
Successfully built cython-bbox


In [11]:
import torch
print(torch.__version__)

2.4.1+cu121


In [12]:
!mkdir /content/AlphaPose/img_dir
!mkdir /content/AlphaPose/output_dir

In [13]:
!ls

alphapose
alphapose.egg-info
build
configs
detector
docs
examples
img_dir
LICENSE
model_files
osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth
output_dir
pretrained_models
README.md
scripts
setup.cfg
setup.py
trackers


In [25]:
# alpha pose through webcam
# !ls
cfg_file='configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml'
trained_model='pretrained_models/simple_res50_256x192.pth'
img_directory='img_dir/white_man_standing.png'
output_directory='output_dir'
# can't open webcam from google collab.
# !python scripts/demo_inference.py --cfg {cfg_file} --checkpoint {trained_model} --outdir examples/res --vis --webcam 0
!python scripts/demo_inference.py --cfg {cfg_file} --checkpoint {trained_model} --image {img_directory} --outdir {output_directory} --pose_track  --save_img



2024-10-16 19:54:09 [DEBUG]: Loaded backend agg version v2.2.
inputpath 
inputlist 
inputimg img_dir/white_man_standing.png
im_names ['white_man_standing.png']
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loading pose model from pretrained_models/simple_res50_256x192.pth...
/content/AlphaPose/scripts/demo_inference.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), 